In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15

import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import math

import sklearn
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 15 seconds


In [3]:
train_data = pd.read_csv('Data/train.csv')
train_data = train_data.drop("id", axis = 1)
def convert_target(x):
    return int(x["target"].split('_')[1])
train_data["target"] = train_data.apply(convert_target, axis = 1)

In [4]:
from sklearn.metrics import log_loss, accuracy_score
import datetime, time
def test_method(method, a_train, a_test, b_train, b_test):
    print method.__class__
    method.fit(a_train, b_train)
    print log_loss(b_train, method.predict_proba(a_train))

def test_methods(methods, a_train, a_test, b_train, b_test):
    for method in methods:
        test_method(method, a_train, a_test, b_train, b_test)

def test_models(data):
    target = data.target
    train = data.drop('target', axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.2, random_state = 42)
    methods = [RandomForestClassifier(n_estimators = 100)]
    test_methods(methods, X_train, X_test, y_train, y_test)

def get_best_params(method, params, data):
    target = data.target
    train = data.drop('target', axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.2, random_state = 42)
    errors_train, errors_test = aggregate_train_test_errors(method, params, X_train, y_train, X_test, y_test, [log_loss], print_flag=True)
    plot_train_test_errors(method, params, errors_train, errors_test)
    
def print_score(model, metric, X_train, y_train, X_test, y_test):
        print("Model: %s Metric: %s On test: %f" % (model.__class__.__name__, metric.__name__, metric(y_test, model.predict_proba(X_test))))
        
def print_scores(model, metrics, X_train, y_train, X_test, y_test, parameter = None):
    for metric in metrics:
        if parameter != None:
            print("%s %s" % (parameter[0], parameter[1]))
            print_score(model, metric, X_train, y_train, X_test, y_test)
        else:
            print_score(model, metric, X_train, y_train, X_test, y_test)
    
def aggregate_train_test_errors(model, params, X_train, y_train, X_test, y_test, metrics = [], print_flag = False):
    errors_train = {key: {'params': params[key], 'values': []} for key in params.keys()}
    errors_test = {key: {'params': params[key], 'values': []}  for key in params.keys()}
    base_parameters = model.get_params()
    start_time = time.time()
    prev_time = start_time
    for key, values in params.iteritems():
        if isinstance(values, list):
            for value in values:
                model.set_params(**{key: value})
                model.fit(X_train, y_train)
                cur_time = time.time()
                from_start = datetime.datetime.fromtimestamp(cur_time - start_time + time.timezone).strftime('%H:%M:%S')
                from_last = datetime.datetime.fromtimestamp(cur_time - prev_time + time.timezone).strftime('%H:%M:%S')
                print from_start + " (" + from_last + ")"
                prev_time = cur_time
                errors_train[key]['values'].append(model.score(X_train, y_train))
                errors_test[key]['values'].append(model.score(X_test, y_test))
                if print_flag == True and metrics:
                    print_scores(model, metrics, X_train, y_train, X_test, y_test, (key, value))
            model.set_params(**base_parameters)
        else:
            raise Exception("Take only list of parameters!")   
    return errors_train, errors_test

def plot_train_test_errors(model, model_params, errors_train, errors_test, save = None):
    fig, axes = plt.subplots(ncols=len(model_params))
    fig.set_size_inches((15,9))
    for ind, param in enumerate(model_params.keys()):
        if isinstance(errors_train[param]['params'][0], str):
            axes[ind].plot(range(len(errors_train[param]['params'])), errors_train[param]['values'], label = 'on train')
            axes[ind].plot(range(len(errors_train[param]['params'])), errors_test[param]['values'], label = 'on test')
            axes[ind].xaxis.set_ticks(range(len(errors_train[param]['params'])), errors_train[param]['params'])
        else:            
            axes[ind].plot(errors_train[param]['params'], errors_train[param]['values'], label = 'on train')
            axes[ind].plot(errors_train[param]['params'], errors_test[param]['values'], label = 'on test')
            axes[ind].xaxis.set_ticks(range(len(errors_train[param]['params'])), errors_train[param]['params'])
        axes[ind].set_title(str(param))
    plt.legend()
    plt.tight_layout()

In [7]:
gb_best_params = {"n_estimators": 600, 
                  "max_depth": 9,
                  "max_features" : 30}

In [5]:
clf = GradientBoostingClassifier(n_estimators=600, max_depth=9, max_features=30, verbose=1)
target = train_data.target
train = train_data.drop('target', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.2, random_state = 42)
clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1       85596.7717          106.18m
         2       73668.7873          106.28m
         3       65173.3394          106.40m
         4       58595.1975          106.41m
         5       53128.5161          106.66m
         6       48680.5411          106.49m
         7       44992.3642          106.76m
         8       41834.8191          106.37m
         9       39104.2283          106.27m
        10       36728.2096          106.53m
        20       23466.2341          106.52m
        30       17478.9074          107.77m
        40       13983.9349          106.08m
        50       11574.0767          104.48m
        60        9824.2811          102.81m
        70        8408.7248          100.95m
        80        7358.1647           98.61m
        90        6514.8144           96.32m
       100        5749.9801           94.32m
       200        2052.7256           74.08m
       300         820.0813           55.51m
       40

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=9, max_features=30, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2, n_estimators=600,
              random_state=None, subsample=1.0, verbose=1,
              warm_start=False)